In [36]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from LSTM_preprocess import *

data = pd.read_csv('reddit_train.csv', usecols=['comments', 'subreddits'])

In [37]:
data.subreddits.value_counts()

Music              3500
Overwatch          3500
nba                3500
europe             3500
baseball           3500
hockey             3500
AskReddit          3500
anime              3500
gameofthrones      3500
wow                3500
conspiracy         3500
leagueoflegends    3500
funny              3500
GlobalOffensive    3500
trees              3500
nfl                3500
movies             3500
soccer             3500
worldnews          3500
canada             3500
Name: subreddits, dtype: int64

In [38]:
num_of_categories = 3500
shuffled = data.reindex(np.random.permutation(data.index))
a = shuffled[shuffled['subreddits'] == 'AskReddit'][:num_of_categories]
b = shuffled[shuffled['subreddits'] == 'GlobalOffensive'][:num_of_categories]
c = shuffled[shuffled['subreddits'] == 'Music'][:num_of_categories]
d = shuffled[shuffled['subreddits'] == 'Overwatch'][:num_of_categories]
e = shuffled[shuffled['subreddits'] == 'anime'][:num_of_categories]
f = shuffled[shuffled['subreddits'] == 'baseball'][:num_of_categories]
g = shuffled[shuffled['subreddits'] == 'canada'][:num_of_categories]
h = shuffled[shuffled['subreddits'] == 'conspiracy'][:num_of_categories]
i = shuffled[shuffled['subreddits'] == 'europe'][:num_of_categories]
j = shuffled[shuffled['subreddits'] == 'funny'][:num_of_categories]
k = shuffled[shuffled['subreddits'] == 'gameofthrones'][:num_of_categories]
l = shuffled[shuffled['subreddits'] == 'hocky'][:num_of_categories]
m = shuffled[shuffled['subreddits'] == 'leagueoflegends'][:num_of_categories]
n = shuffled[shuffled['subreddits'] == 'movies'][:num_of_categories]
o = shuffled[shuffled['subreddits'] == 'nba'][:num_of_categories]
p = shuffled[shuffled['subreddits'] == 'nfl'][:num_of_categories]
q = shuffled[shuffled['subreddits'] == 'soccer'][:num_of_categories]
r = shuffled[shuffled['subreddits'] == 'trees'][:num_of_categories]
s = shuffled[shuffled['subreddits'] == 'worldnews'][:num_of_categories]
t = shuffled[shuffled['subreddits'] == 'wow'][:num_of_categories]

concated = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

In [39]:
#One-hot encode the lab
concated.loc[concated['subreddits'] == 'AskReddit', 'LABEL'] = 0
concated.loc[concated['subreddits'] == 'GlobalOffensive', 'LABEL'] = 1
concated.loc[concated['subreddits'] == 'Music', 'LABEL'] = 2
concated.loc[concated['subreddits'] == 'Overwatch', 'LABEL'] = 3
concated.loc[concated['subreddits'] == 'anime', 'LABEL'] = 4
concated.loc[concated['subreddits'] == 'baseball', 'LABEL'] = 5
concated.loc[concated['subreddits'] == 'canada', 'LABEL'] = 6
concated.loc[concated['subreddits'] == 'conspiracy', 'LABEL'] = 7
concated.loc[concated['subreddits'] == 'europe', 'LABEL'] = 8
concated.loc[concated['subreddits'] == 'funny', 'LABEL'] = 9
concated.loc[concated['subreddits'] == 'gameofthrones', 'LABEL'] = 10
concated.loc[concated['subreddits'] == 'hocky', 'LABEL'] = 11
concated.loc[concated['subreddits'] == 'leagueoflegends', 'LABEL'] = 12
concated.loc[concated['subreddits'] == 'movies', 'LABEL'] = 13
concated.loc[concated['subreddits'] == 'nba', 'LABEL'] = 14
concated.loc[concated['subreddits'] == 'nfl', 'LABEL'] = 15
concated.loc[concated['subreddits'] == 'soccer', 'LABEL'] = 16
concated.loc[concated['subreddits'] == 'trees', 'LABEL'] = 17
concated.loc[concated['subreddits'] == 'worldnews', 'LABEL'] = 18
concated.loc[concated['subreddits'] == 'wow', 'LABEL'] = 19



print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=20)
print(labels[:10])
if 'subreddits' in concated.keys():
    concated.drop(['subreddits'], axis=1)

3879      1
48288    14
18794     5
38245    10
48245    14
5200      1
15582     4
21398     6
27501     7
38543    12
Name: LABEL, dtype: int64
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [40]:
for i in range(len(concated["comments"])):
    concated["comments"][i] = preprocess_text(concated["comments"][i])

D:\PROGRAMS\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
n_most_common_words = 20000
max_len = 200
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789', lower=True)
tokenizer.fit_on_texts(concated['comments'].values)
sequences = tokenizer.texts_to_sequences(concated['comments'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 56700 unique tokens.


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.2, random_state=42)

In [43]:
epochs = 1000
emb_dim = 256
batch_size = 256
labels[:2]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [44]:
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

((53200, 200), (53200, 20), (13300, 200), (13300, 20))


In [45]:
model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(128, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(20, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.25,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 256)          5120000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 200, 256)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_3 (Dense)              (None, 20)                2580      
Total params: 5,319,700
Trainable params: 5,319,700
Non-trainable params: 0
_________________________________________________________________
None


D:\PROGRAMS\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 39900 samples, validate on 13300 samples
Epoch 1/1000
39900/39900 [==============================] - ETA: 7:14 - loss: 2.9993 - acc: 0.046 - ETA: 6:51 - loss: 2.9931 - acc: 0.044 - ETA: 6:37 - loss: 2.9961 - acc: 0.044 - ETA: 6:40 - loss: 2.9960 - acc: 0.046 - ETA: 6:39 - loss: 2.9960 - acc: 0.047 - ETA: 6:42 - loss: 2.9979 - acc: 0.045 - ETA: 6:40 - loss: 2.9987 - acc: 0.046 - ETA: 6:40 - loss: 2.9989 - acc: 0.046 - ETA: 6:37 - loss: 2.9975 - acc: 0.051 - ETA: 6:31 - loss: 2.9981 - acc: 0.050 - ETA: 6:26 - loss: 2.9977 - acc: 0.051 - ETA: 6:21 - loss: 2.9973 - acc: 0.049 - ETA: 6:15 - loss: 2.9975 - acc: 0.049 - ETA: 6:12 - loss: 2.9968 - acc: 0.051 - ETA: 6:10 - loss: 2.9966 - acc: 0.052 - ETA: 6:08 - loss: 2.9965 - acc: 0.051 - ETA: 6:05 - loss: 2.9967 - acc: 0.051 - ETA: 6:02 - loss: 2.9964 - acc: 0.050 - ETA: 5:59 - loss: 2.9956 - acc: 0.051 - ETA: 5:56 - loss: 2.9950 - acc: 0.052 - ETA: 5:55 - loss: 2.9953 - acc: 0.052 - ETA: 5:54 - loss: 2.9950 - acc: 0.051 - ETA: 5:51 

39900/39900 [==============================] - ETA: 6:34 - loss: 2.8987 - acc: 0.093 - ETA: 6:29 - loss: 2.9047 - acc: 0.097 - ETA: 6:26 - loss: 2.9066 - acc: 0.102 - ETA: 6:24 - loss: 2.9027 - acc: 0.101 - ETA: 6:21 - loss: 2.9008 - acc: 0.100 - ETA: 6:18 - loss: 2.8986 - acc: 0.101 - ETA: 6:15 - loss: 2.8972 - acc: 0.099 - ETA: 6:12 - loss: 2.8973 - acc: 0.102 - ETA: 6:09 - loss: 2.8967 - acc: 0.100 - ETA: 6:06 - loss: 2.8944 - acc: 0.103 - ETA: 6:03 - loss: 2.8953 - acc: 0.104 - ETA: 6:01 - loss: 2.8948 - acc: 0.106 - ETA: 5:58 - loss: 2.8956 - acc: 0.106 - ETA: 5:55 - loss: 2.8951 - acc: 0.105 - ETA: 5:52 - loss: 2.8953 - acc: 0.106 - ETA: 5:50 - loss: 2.8951 - acc: 0.106 - ETA: 5:47 - loss: 2.8951 - acc: 0.106 - ETA: 5:44 - loss: 2.8951 - acc: 0.105 - ETA: 5:41 - loss: 2.8934 - acc: 0.107 - ETA: 5:38 - loss: 2.8930 - acc: 0.107 - ETA: 5:36 - loss: 2.8918 - acc: 0.107 - ETA: 5:33 - loss: 2.8901 - acc: 0.107 - ETA: 5:30 - loss: 2.8891 - acc: 0.108 - ETA: 5:28 - loss: 2.8888 - acc: 0

39900/39900 [==============================] - ETA: 6:36 - loss: 2.6171 - acc: 0.218 - ETA: 6:26 - loss: 2.5627 - acc: 0.222 - ETA: 6:24 - loss: 2.5509 - acc: 0.225 - ETA: 6:37 - loss: 2.5315 - acc: 0.222 - ETA: 6:33 - loss: 2.5147 - acc: 0.228 - ETA: 6:29 - loss: 2.5157 - acc: 0.225 - ETA: 6:28 - loss: 2.5225 - acc: 0.218 - ETA: 6:27 - loss: 2.5209 - acc: 0.220 - ETA: 6:30 - loss: 2.5142 - acc: 0.218 - ETA: 6:27 - loss: 2.5040 - acc: 0.218 - ETA: 6:26 - loss: 2.5068 - acc: 0.215 - ETA: 6:22 - loss: 2.5117 - acc: 0.213 - ETA: 6:19 - loss: 2.5156 - acc: 0.212 - ETA: 6:15 - loss: 2.5187 - acc: 0.210 - ETA: 6:11 - loss: 2.5181 - acc: 0.209 - ETA: 6:07 - loss: 2.5159 - acc: 0.210 - ETA: 6:03 - loss: 2.5159 - acc: 0.210 - ETA: 6:01 - loss: 2.5162 - acc: 0.210 - ETA: 5:59 - loss: 2.5160 - acc: 0.211 - ETA: 5:57 - loss: 2.5154 - acc: 0.212 - ETA: 5:55 - loss: 2.5165 - acc: 0.211 - ETA: 5:53 - loss: 2.5153 - acc: 0.210 - ETA: 5:52 - loss: 2.5130 - acc: 0.210 - ETA: 5:52 - loss: 2.5121 - acc: 0

...

39900/39900 [==============================] - ETA: 6:46 - loss: 1.2163 - acc: 0.617 - ETA: 6:37 - loss: 1.1717 - acc: 0.644 - ETA: 6:33 - loss: 1.1735 - acc: 0.649 - ETA: 6:28 - loss: 1.1899 - acc: 0.636 - ETA: 6:27 - loss: 1.2149 - acc: 0.628 - ETA: 6:24 - loss: 1.2270 - acc: 0.625 - ETA: 6:20 - loss: 1.2439 - acc: 0.628 - ETA: 6:18 - loss: 1.2404 - acc: 0.629 - ETA: 6:19 - loss: 1.2488 - acc: 0.625 - ETA: 6:18 - loss: 1.2609 - acc: 0.623 - ETA: 6:18 - loss: 1.2607 - acc: 0.621 - ETA: 6:16 - loss: 1.2546 - acc: 0.623 - ETA: 6:13 - loss: 1.2560 - acc: 0.622 - ETA: 6:10 - loss: 1.2615 - acc: 0.622 - ETA: 6:07 - loss: 1.2625 - acc: 0.621 - ETA: 6:04 - loss: 1.2619 - acc: 0.620 - ETA: 6:01 - loss: 1.2691 - acc: 0.619 - ETA: 6:00 - loss: 1.2750 - acc: 0.618 - ETA: 5:58 - loss: 1.2769 - acc: 0.615 - ETA: 5:55 - loss: 1.2798 - acc: 0.613 - ETA: 5:52 - loss: 1.2821 - acc: 0.611 - ETA: 5:49 - loss: 1.2827 - acc: 0.610 - ETA: 5:48 - loss: 1.2744 - acc: 0.613 - ETA: 5:46 - loss: 1.2713 - acc: 0

39900/39900 [==============================] - ETA: 6:55 - loss: 1.3535 - acc: 0.605 - ETA: 7:34 - loss: 1.2422 - acc: 0.615 - ETA: 7:56 - loss: 1.2757 - acc: 0.610 - ETA: 7:42 - loss: 1.2920 - acc: 0.603 - ETA: 7:25 - loss: 1.2631 - acc: 0.610 - ETA: 7:12 - loss: 1.2735 - acc: 0.612 - ETA: 7:01 - loss: 1.2740 - acc: 0.615 - ETA: 6:53 - loss: 1.2596 - acc: 0.618 - ETA: 6:46 - loss: 1.2627 - acc: 0.616 - ETA: 6:45 - loss: 1.2670 - acc: 0.614 - ETA: 6:48 - loss: 1.2594 - acc: 0.616 - ETA: 6:48 - loss: 1.2571 - acc: 0.614 - ETA: 6:41 - loss: 1.2477 - acc: 0.618 - ETA: 6:36 - loss: 1.2537 - acc: 0.615 - ETA: 6:31 - loss: 1.2489 - acc: 0.616 - ETA: 6:26 - loss: 1.2451 - acc: 0.617 - ETA: 6:23 - loss: 1.2467 - acc: 0.617 - ETA: 6:18 - loss: 1.2331 - acc: 0.620 - ETA: 6:13 - loss: 1.2323 - acc: 0.621 - ETA: 6:09 - loss: 1.2309 - acc: 0.621 - ETA: 6:05 - loss: 1.2359 - acc: 0.620 - ETA: 6:01 - loss: 1.2321 - acc: 0.621 - ETA: 5:57 - loss: 1.2307 - acc: 0.623 - ETA: 5:54 - loss: 1.2291 - acc: 0

39900/39900 [==============================] - ETA: 7:18 - loss: 1.2865 - acc: 0.589 - ETA: 7:24 - loss: 1.2298 - acc: 0.632 - ETA: 7:19 - loss: 1.2181 - acc: 0.628 - ETA: 7:13 - loss: 1.2227 - acc: 0.631 - ETA: 7:07 - loss: 1.2115 - acc: 0.636 - ETA: 7:02 - loss: 1.2204 - acc: 0.635 - ETA: 6:58 - loss: 1.2096 - acc: 0.636 - ETA: 6:55 - loss: 1.1894 - acc: 0.643 - ETA: 6:52 - loss: 1.1932 - acc: 0.641 - ETA: 6:50 - loss: 1.1959 - acc: 0.639 - ETA: 6:48 - loss: 1.1963 - acc: 0.641 - ETA: 6:45 - loss: 1.1922 - acc: 0.643 - ETA: 6:42 - loss: 1.1862 - acc: 0.647 - ETA: 6:40 - loss: 1.1818 - acc: 0.649 - ETA: 6:37 - loss: 1.1818 - acc: 0.649 - ETA: 6:35 - loss: 1.1856 - acc: 0.648 - ETA: 6:32 - loss: 1.1903 - acc: 0.647 - ETA: 6:30 - loss: 1.1858 - acc: 0.648 - ETA: 6:28 - loss: 1.1913 - acc: 0.647 - ETA: 6:26 - loss: 1.1841 - acc: 0.650 - ETA: 6:23 - loss: 1.1866 - acc: 0.649 - ETA: 6:21 - loss: 1.1832 - acc: 0.650 - ETA: 6:19 - loss: 1.1844 - acc: 0.652 - ETA: 6:19 - loss: 1.1842 - acc: 0

39900/39900 [==============================] - ETA: 6:35 - loss: 1.1302 - acc: 0.652 - ETA: 6:45 - loss: 1.2073 - acc: 0.632 - ETA: 6:41 - loss: 1.1512 - acc: 0.641 - ETA: 6:36 - loss: 1.1360 - acc: 0.643 - ETA: 6:33 - loss: 1.1210 - acc: 0.650 - ETA: 6:34 - loss: 1.1067 - acc: 0.651 - ETA: 6:35 - loss: 1.1345 - acc: 0.644 - ETA: 6:36 - loss: 1.1321 - acc: 0.645 - ETA: 6:36 - loss: 1.1362 - acc: 0.650 - ETA: 6:35 - loss: 1.1396 - acc: 0.651 - ETA: 6:32 - loss: 1.1395 - acc: 0.653 - ETA: 6:30 - loss: 1.1255 - acc: 0.657 - ETA: 6:28 - loss: 1.1281 - acc: 0.656 - ETA: 6:26 - loss: 1.1277 - acc: 0.657 - ETA: 6:24 - loss: 1.1260 - acc: 0.657 - ETA: 6:22 - loss: 1.1293 - acc: 0.656 - ETA: 6:19 - loss: 1.1406 - acc: 0.653 - ETA: 6:17 - loss: 1.1390 - acc: 0.654 - ETA: 6:14 - loss: 1.1481 - acc: 0.651 - ETA: 6:11 - loss: 1.1508 - acc: 0.651 - ETA: 6:08 - loss: 1.1478 - acc: 0.652 - ETA: 6:06 - loss: 1.1537 - acc: 0.649 - ETA: 6:03 - loss: 1.1606 - acc: 0.646 - ETA: 6:00 - loss: 1.1650 - acc: 0

39900/39900 [==============================] - ETA: 6:24 - loss: 1.0476 - acc: 0.648 - ETA: 6:25 - loss: 1.0957 - acc: 0.648 - ETA: 6:22 - loss: 1.1345 - acc: 0.653 - ETA: 6:21 - loss: 1.1436 - acc: 0.650 - ETA: 6:19 - loss: 1.1613 - acc: 0.643 - ETA: 6:16 - loss: 1.1293 - acc: 0.655 - ETA: 6:11 - loss: 1.1494 - acc: 0.650 - ETA: 6:14 - loss: 1.1331 - acc: 0.653 - ETA: 6:12 - loss: 1.1217 - acc: 0.653 - ETA: 6:09 - loss: 1.1112 - acc: 0.654 - ETA: 6:09 - loss: 1.1059 - acc: 0.656 - ETA: 6:06 - loss: 1.1124 - acc: 0.656 - ETA: 6:04 - loss: 1.1283 - acc: 0.652 - ETA: 6:01 - loss: 1.1387 - acc: 0.648 - ETA: 5:59 - loss: 1.1325 - acc: 0.651 - ETA: 5:56 - loss: 1.1269 - acc: 0.652 - ETA: 5:53 - loss: 1.1293 - acc: 0.652 - ETA: 5:51 - loss: 1.1283 - acc: 0.653 - ETA: 5:49 - loss: 1.1338 - acc: 0.653 - ETA: 5:46 - loss: 1.1374 - acc: 0.653 - ETA: 5:43 - loss: 1.1354 - acc: 0.654 - ETA: 5:41 - loss: 1.1358 - acc: 0.654 - ETA: 5:38 - loss: 1.1334 - acc: 0.655 - ETA: 5:35 - loss: 1.1352 - acc: 0

In [12]:
# read the test set
test_data = pd.read_csv('reddit_test.csv', usecols=['id','comments'])
for i in range(len(test_data["comments"])):
    test_data["comments"][i] = preprocess_text(test_data["comments"][i])

test_sequences = tokenizer.texts_to_sequences(test_data['comments'].values)
max_len = 200
test_X = pad_sequences(test_sequences, maxlen=max_len)
prediction = model.predict(test_X)

D:\PROGRAMS\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [20]:
print((prediction[2]))

[1.6603477e-03 1.6893185e-04 6.4626784e-04 6.1592192e-04 9.3740296e-01
 2.2477540e-04 1.1542122e-04 3.2671602e-04 4.6760516e-04 2.7686174e-03
 4.8264828e-02 6.9848693e-06 6.4075837e-05 5.4430682e-03 1.0488226e-04
 7.1348477e-05 7.8151104e-05 4.5965239e-04 6.2732463e-04 4.8216953e-04]


In [ ]:
class_dict ={
    0:'AskReddit',
    1:'GlobalOffensive',
    2:'Music',
    3:'Overwatch',
    4:'anime',
    5:'baseball',
    6:'canada',
    7:'conspiracy',
    8:'europe',
    9:'funny',
    10:'gameofthrones',
    11:'hocky',
    12:'leagueoflegends',
    13:'movies',
    14:'nba', 
    15:'nfl', 
    16:'soccer',
    17:'trees',
    18:'worldnews',
    19:'wow'}

In [ ]:
# save the model to local
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)